# Creating Vector Database
## Load Configurations & Libraries


In [1]:
# # If we run this machine on GPU machine it will be faster.
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -Uq sentence-transformers

In [1]:
# @title Load Configuraiton 
import hbqaconfig
conf = hbqaconfig.setEnv('local')
# for k,v in conf.items(): print (k,":",v)

### Selecting and Loading Embedding Model.
- Different models are created on different kind of corpus
- Different vector size
- Some are normailzied, in that case dot product and cosine is same. Dot is less costly

In [62]:
!pip install huggingface
import huggingface

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [3]:
# https://www.sbert.net/docs/pretrained_models.html

#250MB, multi-qa-distilbert-cos-v1',  Max Sequence Length:	512, Dimensions:768, Normalized Embeddings:	true
#80MB, all-MiniLM-L6-v2, Max Sequence Length:	256, Dimensions:	384, Normalized Embeddings:	true
##290MB, all-distilroberta-v1, Max Sequence Length:	512, Dimensions:	768, Normalized Embeddings:	true
#420MB, all-mpnet-base-v2, Max Sequence Length:	384, Dimensions:	768, Normalized Embeddings:	true
##1.36GB, all-roberta-large-v1, Max Sequence Length:	256, Dimensions: 1024, Normalized Embeddings:	true

# embmodelname = 'multi-qa-distilbert-cos-v1'
# embmodelname = 'all-MiniLM-L6-v2'
embmodelname = 'all-distilroberta-v1'
embmodelname = 'multi-qa-mpnet-base-dot-v1'
embmodelname =  'all-roberta-large-v1'

embmodelname1 = "_" + embmodelname

In [3]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

c:\Users\hari_\anaconda3\envs\hbqa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#util.dot_score, util.cos_sim

embmodel = SentenceTransformer(embmodelname)

# Util Functions

In [8]:
# When some function is decorated with this function. It will show the time to execute that function.
# we will use this calculate the time for embedding.
def get_time(func):
    import time
    def wrapper():
        start_time = time.time()

        result = func()

        end_time = time.time()
        time_spent=end_time-start_time

        print(f"Time Taken to Process {time_spent}")
        return result

    return wrapper

In [15]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

DEVICE

device(type='cuda')

In [ ]:
def select_model(num):
    emb_modelshortlist = ['distilbert','minilm','distilroberta','mpnet','roberta']

    emb_modellist = ['multi-qa-distilbert-cos-v1',
                'all-MiniLM-L6-v2',
                'all-distilroberta-v1',
                'multi-qa-mpnet-base-dot-v1',
                'all-roberta-large-v1']

    embmodelname = emb_modellist[num]
    embmodelshort = emb_modelshortlist[num]
    embmodelname1 = "_" + embmodelname

    print (embmodelname,'\t',embmodelshort,'\t', embmodelname1)
    return embmodelname, embmodelshort, embmodelname1
	

# Creat Question & Answer Embedding

## Load Question & Answer Dataset

In [23]:
# df_hbqa_auto_qa = pd.read_csv(datapath+"06.1-HBQA_Auto_with_Chunk.csv")
df_hbqa_manual_qa = pd.read_csv(conf['QAGS_FOLDER']+"06-HBQA_Manual_with_Chunk.csv")


In [24]:
print(df_hbqa_manual_qa.shape)
df_hbqa_manual_qa.head(2)

(1102, 11)


,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk
0,0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809
1,1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852


In [25]:
@get_time
def create_sent_vector():

    Ques_Sentences = df_hbqa_manual_qa.Question.tolist()
    Ques_Embeddings = embmodel.encode(Ques_Sentences)

    Ans_Sentences = df_hbqa_manual_qa.Ref_Answer.tolist()
    Ans_Embeddings = embmodel.encode(Ans_Sentences)

    print ("Quest Embedding: ", Ques_Embeddings.shape,"Answer Embedding: ", Ans_Embeddings.shape)
    # return (Ques_Embeddings, Ans_Embeddings)

    return (Ques_Embeddings, Ans_Embeddings)

In [26]:
print(embmodelname)
Ques_Embeddings, Ans_Embeddings = create_sent_vector()

all-roberta-large-v1
Quest Embedding:  (1102, 1024) Answer Embedding:  (1102, 1024)
Time Taken to Process 21.41625666618347


In [27]:
# Convert These embedding into torch tensor and load into available DEVICE
QuesVectors= torch.tensor(Ques_Embeddings, dtype=torch.float).to(DEVICE)
AnsVectors= torch.tensor(Ans_Embeddings, dtype=torch.float).to(DEVICE)

In [28]:
# Flatten the tensors into 1D arrays and save
df_hbqa_manual_qa['QuesVector'+embmodelname1] = QuesVectors.tolist()
df_hbqa_manual_qa['AnsVector'+embmodelname1] = AnsVectors.tolist()

In [29]:
# df_hbqa_Vector = df_hbqa_manual_qa.copy()

In [30]:
# df_hbqa_Vector.drop(columns = ['WordsInQues','WordsInAns'], axis=1, inplace=True)

In [31]:
print(df_hbqa_manual_qa.shape)
df_hbqa_manual_qa.head(2)

(1102, 13)


,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk,QuesVector_all-roberta-large-v1,AnsVector_all-roberta-large-v1
0,0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809,"[0.000841788831166923, 0.042884085327386856, -...","[0.021666517481207848, 0.04146146401762962, -0..."
1,1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852,"[0.009102409705519676, 0.0005071111372672021, ...","[-0.008400637656450272, 0.013107581995427608, ..."


In [32]:
#Save question answer vector on Harddisk

df_hbqa_manual_qa.to_csv(conf['QAGS_FOLDER'] + '04.3-HBQA_QA_Vector'+embmodelname1+'.csv')

# Check the alignment of Questions and Answer

## Run these cells 5 time while change n value 0,1,2,3,4

In [102]:
n=4
embmodelname, embmodelshort, embmodelname1 = select_model(n)
filename = '04.3-HBQA_QA_Vector'+embmodelname1+'.csv'
print(filename)
df = pd.read_csv(conf['QAGS_FOLDER'] + filename)

all-roberta-large-v1 	 roberta 	 _all-roberta-large-v1
07.2-HBQA_QA_Vector_all-roberta-large-v1.csv


In [68]:
# df.head(2)

In [103]:
df[['Ques_Id', 'QuesVector'+embmodelname1,	'AnsVector'+embmodelname1]][:2]


,Ques_Id,QuesVector_all-roberta-large-v1,AnsVector_all-roberta-large-v1
0,0,"[0.000841788831166923, 0.042884085327386856, -...","[0.021666517481207848, 0.04146146401762962, -0..."
1,1,"[0.009102409705519676, 0.0005071111372672021, ...","[-0.008400637656450272, 0.013107581995427608, ..."


In [104]:
ques_ans_cos = []
for i in df.index:
  quesvec = df.loc[i,'QuesVector'+embmodelname1]
  ansvec = df.loc[i,'AnsVector'+embmodelname1]

  cos = util.cos_sim(ast.literal_eval(quesvec), ast.literal_eval(ansvec)).numpy()[0][0]

  ques_ans_cos.append(cos)
  print(cos)



0.7576374
0.70311856
0.65739226
0.64778477
0.85956824
0.82034135
0.7001169
0.6771966
0.7628859
0.68177223
0.78931224
0.64798546
0.517642
0.78892064
0.70680964
0.78475565
0.646376
0.8505906
0.55179393
0.71393687
0.8192592
0.8673236
0.71014255
0.7624631
0.6615807
0.8401385
0.7860886
0.7530787
0.7326288
0.8346249
0.80706096
0.8174927
0.70083606
0.7318264
0.7163131
0.7684248
0.7931408
0.8091314
0.8006215
0.6747469
0.75076425
0.51405257
0.5754106
0.6615684
0.8200116
0.8293636
0.8080141
0.6434109
0.8280064
0.8566126
0.7607194
0.8036401
0.8385169
0.69325215
0.69738466
0.7708157
0.7073339
0.7004595
0.8255744
0.78408086
0.76892483
0.73911077
0.72692037
0.7215806
0.7906822
0.66121304
0.7196022
0.8656324
0.76258075
0.77050984
0.7652155
0.68921417
0.81863517
0.78534454
0.8095366
0.5808427
0.63370717
0.7072624
0.67246485
0.6882262
0.81138563
0.7216852
0.8108847
0.84078693
0.65942
0.5917322
0.7788893
0.76719403
0.8071332
0.73952395
0.7198832
0.66265
0.8090793
0.8261365
0.69639826
0.6617503
0.565879


In [105]:
df['Ques_Ans_Cos'] = ques_ans_cos

In [106]:
filename = conf['QAGS_FOLDER'] + '04.3-HBQA_Cosine' + embmodelname1 + '.csv'
print(filename)
df[['Ques_Id', 'QuesVector'+embmodelname1 ,	'AnsVector'+embmodelname1,'Ques_Ans_Cos']].to_csv(filename,index=None)

H:\My Drive\HBQA\Data\07.2-HBQA_Cosine_all-roberta-large-v1.csv


In [107]:
# Ques_Ans_Cos_Dic={}
Ques_Ans_Cos_Dic[embmodelname] = df['Ques_Ans_Cos'].mean()

Ques_Ans_Cos_Dic

{'multi-qa-distilbert-cos-v1': 0.77918875,
 'all-MiniLM-L6-v2': 0.77996945,
 'all-distilroberta-v1': 0.70968866,
 'multi-qa-mpnet-base-dot-v1': 0.7660101,
 'all-roberta-large-v1': 0.7311594}

# Run this last

In [113]:
pd.DataFrame(Ques_Ans_Cos_Dic.items(),columns=['Model_Name','Cosine']).to_csv(datapath+'31.6-All-Model-QA-Cosine.csv', index=None)
# pd.DataFrame(Ques_Ans_Cos_Dic.items(),columns=['Model_Name','Cosine']).to_markdown(datapath+'31.6-All-Model-QA-Cosine.md', index=None)
pd.DataFrame(Ques_Ans_Cos_Dic.items(),columns=['Model_Name','Cosine'])

,Model_Name,Cosine
0,multi-qa-distilbert-cos-v1,0.779189
1,all-MiniLM-L6-v2,0.779969
2,all-distilroberta-v1,0.709689
3,multi-qa-mpnet-base-dot-v1,0.766010
4,all-roberta-large-v1,0.731159
